In [12]:
from sklearn.feature_selection import SelectKBest, f_classif, chi2, f_regression
import pickle
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel, SelectPercentile
import csv
import pandas as pd 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import svm
import numpy as np
from sklearn.tree import DecisionTreeRegressor

### Step 1-Loading and Checking Data of Smaller Dataset ###

In [13]:
madelon_traindf = pd.read_csv('madelon_train.data', sep = ' ', header=None)

In [14]:
madelon_traindf.shape

(2000, 501)

In [15]:
madelon_traindf.drop(labels=[500], inplace=True, axis=1)

In [16]:
madelon_traindf.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,485,477,537,479,452,471,491,476,475,473,...,477,481,477,485,511,485,481,479,475,496
1,483,458,460,487,587,475,526,479,485,469,...,463,478,487,338,513,486,483,492,510,517
2,487,542,499,468,448,471,442,478,480,477,...,487,481,492,650,506,501,480,489,499,498
3,480,491,510,485,495,472,417,474,502,476,...,491,480,474,572,454,469,475,482,494,461
4,484,502,528,489,466,481,402,478,487,468,...,488,479,452,435,486,508,481,504,495,511


In [17]:
madelon_valid = pd.read_csv('madelon_valid.data', sep= ' ', header=None)
madelon_valid.drop([500], inplace=True, axis=1)
#madelon_valid

In [18]:
madelon_testdf = pd.read_csv('madelon_test.data', sep= ' ', header=None)


In [19]:
madelon_testdf.drop([500], inplace=True, axis=1)


In [20]:
#need to change the label column name, otherwise it gives me two columns named 0 so try to index it would not end well
madelon_trainlablsdf = pd.read_csv('madelon_train.labels', sep=' ', header=None)
madelon_trainlablsdf.rename(index=None, columns={0:'label'}, inplace=True)
#madelon_trainlablsdf

In [23]:
madelon_validlabelsdf = pd.read_csv('madelon_valid.labels', sep=' ', header=None)


In [24]:
madelon_valid1 = pd.concat([madelon_validlabelsdf, madelon_valid], axis=1)
#madelon_valid1

In [22]:
#madelon_valid1

In [25]:
madelon_traindf1 = pd.concat([madelon_trainlablsdf, madelon_traindf], axis=1)


Taking 10% percent sample to test the models on.

In [26]:
#madelon_traindf1.info()
train_set = madelon_traindf1.sample(200) #replace=True)
train_set2 = madelon_traindf1.sample(200)
train_set3 = madelon_traindf1.sample(200)

In [15]:
#train_set3.info()

In [27]:
test_set = madelon_testdf.sample(200)


In [28]:
valid_set = madelon_valid1.sample(200)

In [18]:
#target_valid = valid_set['label']
#target_valid

In [29]:
values = madelon_traindf.columns.values

In [30]:
target = train_set['label']
X = train_set.drop(['label'], axis=1)
Z = test_set

## Benchmark Models ##

Starting with benchmark models to get a basic idea of how each model does with the data.

I am using pipelines to make sure to simplify the process. 

After fitting the data to the model, I then scored the model to see how well it preformed. 

The Madelon Set that was import, included a train and test set already, so I need not need to do a train test split.

The label data that was imported is target that is used in the all the models to see the correlation to the features in the dataset to that of the target/label. 

### Logistic Regression ###

In [31]:
#Train dataset
logistic_pipeline = make_pipeline(StandardScaler(), LogisticRegression())

In [32]:
logistic = logistic_pipeline.fit(X, target)
logistictrain_score = logistic.score(X,target)

In [33]:
logistictrain_score

1.0

In [94]:
#Test dataset
logistic_pipeline2 = make_pipeline(StandardScaler(), LogisticRegression())
logistic2 = logistic_pipeline2.fit(Z, target)
logisticttest_score = logistic2.score(Z,target)

### KNeighbors Regressor

In [64]:
#going with more neighbors have me a lower score
knearest_pipeline = make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2))
knearest_pipeline2 = make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2))
knearest_pipeline.fit(X,target)
knearest_pipeline2.fit(Z,target)


Pipeline(steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('kneighborsregressor', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=2, p=2,
          weights='uniform'))])

In [67]:
knearesttrain_score = knearest_pipeline.score(X,target)
knearesttest_score2 = knearest_pipeline2.score(Z,target)
knearesttrain_score

0.47089947089947093

In [68]:
knearesttest_score2

0.51758481170245885

In [82]:
pca_pipeline = make_pipeline(StandardScaler(), PCA(n_components=195))
pca_pipeline2 = make_pipeline(StandardScaler(), PCA(n_components=195))
#train dataset fit
pca_pipeline.fit(X)
#test dataset fit
pca_pipeline2.fit(Z)


Pipeline(steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=195, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False))])

In [83]:
pca_score_train = pca_pipeline.score(X,target)
pca_score_train

-444.94302385223165

In [84]:
pca_score_test = pca_pipeline2.score(Z,target)
pca_score_test

-440.06923881013995

### Support Vector Classification

In [85]:
svc = svm.SVC(C=100)
svc2 = svm.SVC(C=100)
svc_train = svc.fit(X, target)


trainsvc_score = svc_train.score(X,target)
testsvc_score = svc.score(Z,target)

In [86]:
trainsvc_score


1.0

### DecisionTree Regressor ###

In [88]:
#max_depth=22, min_samples_leaf=3, max_leaf_nodes=100
regressor = DecisionTreeRegressor(max_depth=22, min_samples_leaf=3, max_leaf_nodes=100)
tree_results = regressor.fit(X, target)
score = tree_results.score(X, target)

In [89]:
score

0.84784037071618767

In [90]:
regressor2 = DecisionTreeRegressor(max_depth=22, min_samples_leaf=3, max_leaf_nodes=100)
tree_results = regressor2.fit(Z, target)
test_score = tree_results2.score(Z, target)

In [91]:
test_score

-0.75287892934951706

The scores list creates a dictionary of the scores to be able to view in a dataframe.

In [96]:
score = []

score.append({'logistic train score': logisticttest_score}) 
score.append({'logistic test score': logistictrain_score}) 
score.append({'PCA train score': pca_score_test})
score.append({'PCA test score':pca_score_train})
score.append({'Knearest score train': knearesttrain_score})
score.append({'Kneareat score test': knearesttest_score2})
score.append({'SVC score train': trainsvc_score})
score.append({'SVC score test': testsvc_score})
score.append({'DecisionTree Regressor score test': score})
score.append({'DecisionTree Regressor score test': test_score})

In [104]:
score

[{'logistic train score': 1.0},
 {'logistic test score': 1.0},
 {'PCA train score': -440.06923881013995},
 {'PCA test score': -444.94302385223165},
 {'Knearest score train': 0.47089947089947093},
 {'Kneareat score test': 0.51758481170245885},
 {'SVC score train': 1.0},
 {'SVC score test': 0.59499999999999997},
 {'DecisionTree Regressor score test': [...]},
 {'DecisionTree Regressor score test': -0.75287892934951706}]

## Feature Selection ##

The function below takes the r2 score for each of the models.
The closer the score get should try to be closer to 1 since have a score of 1 means that model fit perfectly.

Feature Selection using the whole data set.

In [34]:
def calculate_r2_for_feature(df, model): 
    scores_list = []
    for col in df.columns:
        
        X = df.drop(col, axis = 1)
        y = df[col]
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    
    
        regressor = model()
        regressor.fit(X_train, y_train)
    
        score = regressor.score(X_test, y_test)
        scores_list.append(score)
    return scores_list

In [10]:
#Putting the Scores from the function into a dataframe 

In [ ]:
scores = pd.DataFrame(calculate_r2_for_feature(madelon_traindf, DecisionTreeRegressor))
scores = scores.sort_values(0, ascending=False)

In [ ]:
madelon_traindf.head()

The scores below go negative after the first 20 features. This means that once the scores go negative, there is little correlation between the feature and the target.

In [32]:
scores

,0
318,0.966261
28,0.965643
64,0.964777
153,0.964617
433,0.962284
451,0.960907
472,0.958995
241,0.957781
442,0.957728
475,0.957469


In [33]:
top_scores = scores[scores[0] > 0]


In [11]:
##Top 20 features

In [34]:
top_scores

,0
318,0.966261
28,0.965643
64,0.964777
153,0.964617
433,0.962284
451,0.960907
472,0.958995
241,0.957781
442,0.957728
475,0.957469


Feature Selection for the random sample of the whole set, to see if it returns the same top 20 features.

In [100]:
score_sample1 = pd.DataFrame(calculate_r2_for_feature(train_set, DecisionTreeRegressor))

In [101]:
score_sample1.sort_values(0, ascending=False, inplace=True)


In [102]:
score_sample1 = score_sample1.loc[score_sample1[0] > 0]
#score_sample1

In [94]:
sample_set2 = pd.DataFrame(calculate_r2_for_feature(train_set2, DecisionTreeRegressor))

In [95]:
sample_set2.sort_values(0, ascending=False)


,0
319,0.965662
154,0.950892
49,0.950412
106,0.947841
29,0.944132
242,0.937618
434,0.935185
494,0.934149
337,0.933272
282,0.933042


In [106]:
sample2_set = sample_set2.loc[sample_set2[0] > 0]
sample2_set

NameError: name 'sample_set2' is not defined

In [ ]:
train_sample3 = pd.DataFrame(calculate_r2_for_feature(train_set3, DecisionTreeRegressor))

In [89]:
train_sample3.sort_values(0, ascending=False)

,0
65,0.962956
476,0.956670
473,0.953539
494,0.951021
454,0.946070
29,0.945302
319,0.943138
434,0.939604
337,0.935168
379,0.934610


In [90]:
train3_sample = train_sample3.loc[train_sample3[0] > 0]

In [93]:
#gives us 22 main features
#train3_sample
train3_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 29 to 494
Data columns (total 1 columns):
0    20 non-null float64
dtypes: float64(1)
memory usage: 320.0 bytes


## SelectKBest

In [62]:
select_k = SelectKBest(score_func = f_regression, k=200)
k_select = select_k.fit_transform(X, target)
df = pd.DataFrame(k_select)
#df_boolean = pd.DataFrame(select_k.get_support())
#df_boolean = df_boolean[df_boolean[0] != False]


In [3]:
score_selectk = pd.DataFrame(select_k.scores_)
score_selectk.sort_values(0, ascending=False, inplace=True)
top_7 = score_selectk.iloc[0:8].T
top_7

NameError: name 'pd' is not defined

In [88]:
mask = select_k.get_support()
selectkbest = train_set.columns[mask]
#pd.DataFrame(selectkbest)

/opt/conda/lib/python3.6/site-packages/pandas/indexes/base.py:1434: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 501 but corresponding boolean dimension is 500
  result = getitem(key)


In [90]:
best_features = []
column_names = list(train_set.columns.values)

for bool, feature in zip(mask, column_names):
    if bool:
        best_features.append(feature)

In [95]:
support_k = pd.DataFrame(best_features)
#support_k.iloc[20:500]

In [157]:
#select k percentile with ten percent of the over all features
select_k2 = SelectPercentile(score_func = f_regression, percentile=10)
k_select2 = select_k2.fit_transform(X, target)
select_per = pd.DataFrame(select_k2.scores_).sort_values(0, ascending=False, inplace=False)
select_per.iloc[0:8].T

,40,369,285,68,168,41,108,496
0,9.673211,7.585568,6.718517,6.635876,6.281178,5.975656,5.756965,5.639179


## SelectKPercentile

In [136]:
select_k3 = SelectPercentile(score_func = f_regression, percentile=20)
k_select3 = select_k3.fit(train_set, target)
select_per2 = pd.DataFrame(select_k3.scores_).sort_values(0, ascending=False, inplace=False)
select_per2[0:15]
#.iloc[0:6]

,0
41,9.673211
370,7.585568
286,6.718517
69,6.635876
169,6.281178
42,5.975656
109,5.756965
497,5.639179
170,5.302489
96,5.301249


In [141]:
#second
select_k4 = SelectPercentile(score_func = f_regression, percentile=30)
k_select4 = select_k4.fit_transform(train_set, target)
select_per3 = pd.DataFrame(select_k4.scores_).sort_values(0, ascending=False, inplace=False)
select_per3.iloc[0:15]
#.iloc[0:6]

,0
41,9.673211
370,7.585568
286,6.718517
69,6.635876
169,6.281178
42,5.975656
109,5.756965
497,5.639179
170,5.302489
96,5.301249


In [95]:
mask = top_scores.index 

last_df = madelon_traindf1.iloc[mask]      

In [96]:
X = last_df.drop(['label'], axis=1)
target = last_df['label']

In [101]:
regressor2 = DecisionTreeRegressor(max_depth=5, min_samples_leaf=3, max_leaf_nodes=20)
tree_results2 = regressor2.fit(X, target)
score2 = tree_results2.score(X, target)

In [102]:
score2

0.86531986531986527